## Deliverable 2. Create a Customer Travel Destinations Map.

In [29]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bambous Virieux,MU,-20.34,57.76,73.00,83,75,6.93,shower rain
1,1,Qaanaaq,GL,77.48,-69.36,-1.98,87,99,1.25,overcast clouds
2,2,Torbay,CA,47.67,-52.73,30.00,58,75,18.34,broken clouds
3,3,Ales,FR,44.13,4.08,53.01,56,26,8.01,scattered clouds
4,4,Saint-Philippe,RE,-21.36,55.77,73.40,56,75,11.41,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_temp_cities = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_temp_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Hilo,US,19.73,-155.09,80.60,74,90,5.82,overcast clouds
16,16,Port Blair,IN,11.67,92.75,80.89,79,5,4.90,light rain
20,20,Pontianak,ID,-0.03,109.33,76.75,88,100,1.90,moderate rain
23,23,Tchollire,CM,8.40,14.17,81.07,41,22,0.76,few clouds
30,30,Butaritari,KI,3.07,172.79,80.58,73,100,12.39,overcast clouds
34,34,Yendi,GH,9.44,-0.01,85.55,40,100,4.14,overcast clouds
40,40,Vaini,TO,-21.20,-175.20,75.20,83,20,11.41,few clouds
44,44,Morant Bay,JM,17.88,-76.41,77.00,94,75,24.16,shower rain
45,45,Bosaso,SO,11.28,49.18,76.05,56,0,2.98,clear sky
52,52,Bogande,BF,12.97,-0.14,88.52,24,0,8.81,clear sky


In [11]:
# 4a. Determine if there are any empty rows.
preferred_temp_cities.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
clean_df = preferred_temp_cities.dropna()


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,80.60,overcast clouds,19.73,-155.09,
16,Port Blair,IN,80.89,light rain,11.67,92.75,
20,Pontianak,ID,76.75,moderate rain,-0.03,109.33,
23,Tchollire,CM,81.07,few clouds,8.40,14.17,
30,Butaritari,KI,80.58,overcast clouds,3.07,172.79,
34,Yendi,GH,85.55,overcast clouds,9.44,-0.01,
40,Vaini,TO,75.20,few clouds,-21.20,-175.20,
44,Morant Bay,JM,77.00,shower rain,17.88,-76.41,
45,Bosaso,SO,76.05,clear sky,11.28,49.18,
52,Bogande,BF,88.52,clear sky,12.97,-0.14,


In [25]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [40]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Hilo,US,80.60,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
16,Port Blair,IN,80.89,light rain,11.67,92.75,Fortune Resort Bay Island
20,Pontianak,ID,76.75,moderate rain,-0.03,109.33,Aston Pontianak Hotel & Convention Center
23,Tchollire,CM,81.07,few clouds,8.40,14.17,"Auberge ""l'amicale"""
30,Butaritari,KI,80.58,overcast clouds,3.07,172.79,Isles Sunset Lodge
34,Yendi,GH,85.55,overcast clouds,9.44,-0.01,Eyarro Lodge
40,Vaini,TO,75.20,few clouds,-21.20,-175.20,Keleti Beach Resort
44,Morant Bay,JM,77.00,shower rain,17.88,-76.41,Whispering Bamboo Cove Resort
45,Bosaso,SO,76.05,clear sky,11.28,49.18,Red Sea Hotel
52,Bogande,BF,88.52,clear sky,12.97,-0.14,Auberge Wendkouni


In [41]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))